# Операции над массивами NumPy

**Контест**: [413](https://contest.yandex.ru/contest/75039/enter), [414](https://contest.yandex.ru/contest/75037/enter), [415, 416](https://contest.yandex.ru/contest/75040/enter)

На прошлом занятии мы с вами начали знакомство с библиотекой NumPy и рассмотрели возможные способы создания массивов NumPy. Однако, очевидно, что работа с данными не ограничивается исключительно созданием массивов. На этом семинаре мы рассмотрим возможные формы взаимодействия с массивами NumPy и разберемся, как взаимодействовать с ними эффективно.

**Импорт библиотек:**

In [1]:
import numpy as np

## Векторизованные операции

На прошлом семинаре мы рассмотрели производительность Python при работе с массивами данных, и пришли к выводу о его медлительности, связанной с подходом к хранению данных. На самом деле проблема кроется не только в подходе к хранению данных, но и в издержках динамической типизации. Т.е. если мы просто изменим подход к хранению данных без изменения подхода к работе с ними, мы не увидим никакого качественного прироста в быстродействии программы. Мы можем переходить на NumPy, CuPy, PyTorch, на что угодно, но без изменения способа работы с данными, выигрыша в производительности мы не увидим.

Чтобы понять это, давайте рассмотрим простой пример: реализуем функцию, которая для значения каждого элемент входного массива вычисляет обратное ему значение и возвращает новый массив вычисленных значений.

**Необходимые импорты:**

In [2]:
from typing import Sequence
from numbers import Real

**Функция:**

In [3]:
def invert_array(array: Sequence[Real]) -> list[Real]:
    inverted = [
        1. / value if value != 0 else value
        for value in array
    ]

    return inverted

**Тесты:**

In [4]:
array_size = int(1e7)

array_numpy = np.random.uniform(-10, 10, size=array_size)
array_python = array_numpy.tolist()

In [ ]:
%timeit invert_array(array_python)
%timeit invert_array(array_numpy)

In [ ]:
del array_numpy
del array_python

Как мы видим, использование NumPy не просто не решило проблему, а даже замедлило скорость вычислений. Но почему? Как уже было сказано ранее, все дело в динамической типизации и, в частности, в том, как интерпретатор ее реализует. Дело в том, что из-за динамической природы Python интерпретатор имеет возможность определить типы данных конкретных объектов только в момент выполнения кода. Когда мы используем нативные конструкции, типа цикла `for`, для выполнения однотипных операций, в частности,  над однородными данными, помимо описанных вами операций интерпретатору также приходится выполнять проверки типов для каждого элемента однородных данных. Эти проверки не бесплатны, поэтому производительность кода при использовании нативных средств Python для реализации вычислений значительно снижается.

Однако правильное использование NumPy позволяет обойти эти ограничения и воспользоваться фактом однородности данных в массивах для увеличения быстродействия вычислений. NumPy обладает рядом операций, позволяющих обходить ограничения Python, используя исключительно код на С, выходящий за рамки объектов Python. Эти операции отличаются быстродействием и позволяют значительно ускорить ваши вычисления. Такие операции называются **векторизованными операциями**.

### Арифметические операции

Все арифметические операции, с которыми вы работали в "чистом" Python, имеют место и в NumPy, и смысл их остается тем же, что и в обычной алгебре.

In [ ]:
numbers = np.arange(5)

print(
    f"{numbers = }",
    f"numbers + 2 = {numbers + 2};",
    f"numbers - 2 = {numbers - 2};",
    f"numbers * 2 = {numbers * 2};",
    f"numbers / 2 = {numbers / 2};",
    f"numbers % 2 = {numbers % 2};",
    f"numbers // 2 = {numbers // 2};",
    f"numbers ** 2 = {numbers ** 2};",
    sep="\n",
)

In [ ]:
numbers = np.arange(1, 10)

print(
    f"array:\n{numbers}",
    f"array-number operation:\n{numbers * 8}",
    f"array-array operation:\n{numbers - numbers[::-1]}",
    f"2D-array operation:\n{numbers.reshape(3, 3) ** 2}",
    sep="\n\n",
)

Как вы видите, применение векторизованной операции к массиву NumPy аналогично поэлементному применению той же операции в цикле. Только в сравнении с векторизованными операциями, циклы будут работать гораздо медленее. Чтобы в этом убедиться, сравним результаты времени работы нашей функции `invert_array` с аналогичной векторизованной операцией.

In [ ]:
array = np.arange(1, 1 + int(1e6))

In [ ]:
%timeit invert_array(array)
%timeit (1. / array)

In [ ]:
del array

Как можно видеть, разница в быстродействии измеряется несколькими порядками. И чем больше будет размер массива, тем больше будет разница в затрачиваемом на его обработку времени со стороны Python и со стороны NumPy.

Стоит отметить, что все операторные формы векторизованных операций представляют собой адаптеры векторизованных операций в функциональном виде. Приведем ниже таблицу соответствий:

| Операторная форма | Функциональная форма | Описание |
|--|--|--|
| + | np.add | Сложение |
| - | np.substract | Бинарный минус |
| - | np.negative | Унарные минус (эквивалент * (-1)) |
| * | np.multiply | Умножение |
| / | np.divide | "Честное" деление |
| % | np.mod | Вычисление остатка при делении |
| // | np.floor_divide | Вычисление целой части при делении |
| ** | np.power | Возведение в степень |

На первый взгляд наличие векторизованных арифметический операций в функциональном виде кажется избыточным. Однако, не стоит спешить с выводами. Функциональная форма предоставляет вам ряд дополнительных возможностей, к число которых относится оптимизация вычислений по используемой памяти, а также превращение простой векторизованной операции в векторизованное агрегирование. Рассмотрим примеры.

In [ ]:
array1 = np.arange(5)
array2 = np.random.randint(-10, 10, size=array1.shape)

print(
    f"{array1 = }", f"{array2 = }", sep="\n", end="\n\n"
)

np.add(array1, array2, out=array2)

print(
    f"{array1 = }", f"{array2 = }", sep="\n",
)

Итак, в примере выше с помощью специального аргумента `out`, которым обладают все перечисленные в таблице функции, мы явно указали область памяти, в которую будут осуществлена запись результата операции. Это позволяет сэкономить память компьютера, особенно если вы намереваетесь обрабатывать действительно большие массивы данных. Экономия памяти происходит за счет того, что компьютер больше не выделяет дополнительную память для хранения промежуточных вычислений, а сразу записывает результат в указанный буфер. Ведь если бы пример выше был бы реализован более привычным образом:

```python
array2 = array1 + array2
```

То интерпретатор действовал бы следующим образом:
1. Выделил бы область памяти, соответствующую размеру результирующего массива;
2. Вычислил бы правую часть и записал бы результат в буфер;
3. Перекопировал вычисленные значения из буфера в `array2`;

При использовании `out` мы пропускаем шаги с созданием буфера и копированием, вместо этого запись результата происходит сразу в указанную область памяти.

Помимо этого, функциональные версии векторизованных бинарных арифметических операций имеют возможность редуцирования данных. Например: 

In [ ]:
numbers = 2 ** (-np.arange(10, dtype=np.float32))
print(numbers, end="\n\n")

print(
    f"sum reduce: {np.add.reduce(numbers)}",
    f"sub reduce: {np.subtract.reduce(numbers)}",
    f"multiply reduce: {np.multiply.reduce(numbers)}",
    f"divide reduce: {np.divide.reduce(numbers)}",
    sep="\n",
)

В этом случае мы просто применяем указанную операцию к элементам массива до тех пор, пока в массиве не останется одного элемента. Также вы можете вычислять операции над элементами массива кумулятивно - каждый элемент результирующего массива - результат применения операции ко всем предшествующим ему элементам, а также к нему включительно. 

In [ ]:
print(
    f"sum accumulate:\n{np.add.accumulate(numbers)}",
    f"sub accumulate:\n{np.subtract.accumulate(numbers)}",
    f"multiply accumulate:\n{np.multiply.accumulate(numbers)}",
    f"divide accumulate:\n{np.divide.accumulate(numbers)}",
    sep="\n\n",
)

Однако у кумулятивных методов есть более удобные аналоги, которые мы рассмотрим позже.

### Логические операции и булевы маски

Вы также можете использовать логические операторы, в контексте массивов NumPy они будут иметь тот же смысл, что и в булевой алгебре. Результатом применения логической операции к массиву будет новый массив булевых значений, элементы которого - значения выполнения соответствующих операций.

In [ ]:
numbers = np.random.uniform(-5, 5, size=5)

print(
    f"numbers: {numbers}",
    f"numbers < 0 = {numbers < 0}",
    f"numbers > 0 = {numbers > 0}",
    f"numbers == 0 = {numbers == 0}",
    f"numbers <= 0 = {numbers <= 0}",
    f"numbers >= 0 = {numbers >= 0}",
    sep="\n",
)

На первый взгляд может показаться, что это совершенно бесполезные операции, однако это не так. Давайте рассмотрим пару примеров использования булевых массивов.

Напомним, что булев тип данных - это дочерний класс типа данных "целые числа", а соответственно мы можем работать с булевыми значениями, как с целыми числами. Соответственно, с помощью булева массива мы можем, например, подсчитать количество элементов массива, которые удовлетворяют определенному условию:

In [ ]:
print(
    f"negative values amount: {np.add.reduce(numbers < 0)}"
)

Однако у булевых массивов есть и более важная функция. Из прошлого семинара мы помним, что массивы NumPy могут быть проиндексированы булевыми массивами. Соответственно, получаемые таким образом булевы массивы могут быть использованы в качестве так называемой булевой маски, чтобы отфильтровывать определенные элементы массива по заданному условию:

In [ ]:
print(f"negative values: {numbers[numbers < 0]}")

Более того, мы можем использовать сложные условия, комбинируя булевы маски с помощью операторов побитового `И` и побитового `ИЛИ`:

In [ ]:
mask = (0 <= numbers) & (numbers < 3)
print(f"condition values: {numbers[mask]}")

Обращаем отдельное внимание на то, что комбинирование булевых масок осуществляется с помощью побитовых операций `&` и `|`, а не с помощью логических операций `and` и `or`. Также не забывайте использовать скобки для группировки условий.

### Математические операции

Помимо арифметических и логических функций, NumPy имеет достаточно богатую библиотеку арифметических функций: тригонометрических, обратных тригонометрических, гиперболических, обратных гиперболических, экспоненциальных, логарифмических и т.д. Вызовы этих функций очень похожи, поэтому следующая ячейка кода носит скорее демонстрационный характер. При необходимости использования конкретной функции, вы можете обратиться к документации:

In [ ]:
abscissa = np.linspace(0, np.pi, 5)

print(
    f"{abscissa = }",
    f"triganometric:\n{np.sin(abscissa)}",
    f"inverse trigonometric:\n{np.arctan(abscissa)}",
    f"hyperbolic:\n{np.cosh(abscissa)}",
    f"inverse hyperbolic:\n{np.sinh(abscissa)}",
    f"exponential:\n{np.exp(abscissa)}",
    f"logarithmic:\n{np.log2(abscissa[abscissa > 0])}",
    sep="\n\n",
)

В примерах выше описано использование операций с одномерными массивами, однако стоит понимать, что данные функции также могут быть использованы и с массивами произвольной размерности.

## Транслирование (Broadcasting)

В одном из примеров выше мы видели, что аргументами векторизованных арифметических операций могут выступать два массива NumPy одинаковой размерности. Однако одна из ключевых особенностей NumPy заключается в том, что массивы не обязательно должны иметь одинаковые размеры. Проиллюстрируем это утверждение примером:

In [ ]:
lhs = np.arange(9).reshape(3, 3)
rhs = np.arange(3)

print(
    f"lhs:\n{lhs}",
    f"rhs:\n{rhs}",
    f"sum:\n{lhs + rhs}",
    sep="\n\n",
)

Первое, что появляется в голове у неподготовленного человека, увидевшего нечто подобное - это вопрос: почему так? Почему этот кусок кода не возбудил ошибку, по типу `Несовместимые размеры массивов`? На самом деле размеры операндов совместимы, просто NumPy трактует совместимость размеров несколько шире банального равенства значений атрибутов `shape`. Проведение операций над массивами разных, но совместимых размеров называется транслированием (broadcasting). Однако это не значит, что мы можем использовать в качестве операндов одной операции массивы абсолютно любых форм, потому что механизм транслирования имеет свои ограничения и подчиняется следующим правилам:

- Если размерности (`ndim`) двух массивов отличаются, форма (`shape`) массива с меньшей размерностью дополняется единицей с левой стороны:
    ```concole
    1. lhs.ndim != rhs.ndim
    2. lhs.ndim > rhs.ndim
    3. rhs.shape == (3, ) -> rhs.shape == (1, 3)
       [0 1 2] -> [[0 1 2]]
    ```
- Если форма двух массивов не совпадает в каком-то измерении, массив с формой, равной 1 в этом измерении, растягивается вплоть до соответствия форме второго массива:
    ```console
    4. lhs.shape[0] != rhs.shape[0]
    5. rhs.shape == (1, 3) -> rhs.shape == (3, 3)
       [[0 1 2]] -> [[0 1 2]
                     [0 1 2]
                     [0 1 2]]
    ```
- Если в каком-то измерении размеры массивов различаются и ни одно значение измерения не равно 1, генерируется ошибка. В нашем примере этот случай не реализуется.

Наглядно суть транслирования можно понять с помощью следующей иллюстрации:

![broadcasting](./images/broadcasting.png)

## Практика 1. Расстояния между точками прямой

Дан некоторый набор точек $X$, расположенных на прямой: $X \sub \R$. Необходимо найти значения расстояний для каждой пары точек из данного множества $X$. Вычисленные расстояния необходимо представить в виде матрицы $D$ размером $N \times N, N = |X|$, такой, что $D_{ij} = |x_i - x_j| \forall x_i, x_j \in X$.

*Входные данные*:
- `points` - np.ndarray, множество точек прямой, для которых необходимо вычислить попарные расстояния.

*Выходные данные*:
- двумерные массив типа np.ndarray - попарные расстояния между выходных точек. Значению элемента `result[i][j]` соответствует расстояние между точкой `points[i]` и точкой `points[j]`.

**Решение**

In [ ]:
def get_mutual_distances_vectorized(points: np.ndarray) -> np.ndarray:
    # код преподавателя
    return np.array([], dtype=points.dtype)

**Тестирование**:

In [22]:
points = np.arange(5)
distances_expected = np.array(
    [
        [0, 1, 2, 3, 4],
        [1, 0, 1, 2, 3],
        [2, 1, 0, 1, 2],
        [3, 2, 1, 0, 1],
        [4, 3, 2, 1, 0],
    ],
    dtype=points.dtype,
)

distances = get_mutual_distances_vectorized(points)

assert np.allclose(distances, distances_expected)

## Практика 2. Полярные координаты: путешествие туда и обратно

Необходимо реализовать две функции: функцию перевода координат из двухмерной прямоугольной декартовой системы координат в полярные координаты и функцию перевода из полярных координат в двухмерную прямоугольную декартову систему координат.

### Перевод из декартовых координат в полярные

*Входные данные*:
- `abscissa` - np.ndarray, абсциссы точек;
- `ordinates` - np.ndarray, ординаты точек;

*Выходные данные*:
- Кортеж (`tuple`) из двух элементов, каждый элемент - np.ndarray. Первый элемент `tuple` - массив расстояний, второй элемент `tuple` - массив углов в диапазоне от $[-\pi, \pi]$.

*Сторонние эффекты*:
- Если количество элементов во входных массивах `abscissa` и `ordinates` отличаются, необходимо возбудить исключение `ShapeMismatchError`.

*Замечания*:
- Гарантируется, что на вход подаются непустые одномерные массивы чисел с плавающей точкой.
- Предполагаем, что для перевода в полярные координаты используются следующие формулы:
    $$x = r*cos(\phi); y = r*sin(\phi)$$

### Перевод из полярных координат в декартовы

*Входные данные*:
- `distances` - np.ndarray, массив расстояний;
- `angles` - np.ndarray, массив углов в диапазоне $[-\pi, \pi]$;

*Выходные данные*:
- Кортеж (`tuple`) из двух элементов, каждый элемент - np.ndarray. Первый элемент `tuple` - массив абсцисс, второй элемент `tuple` - массив ординат.

*Сторонние эффекты*:
- Если количество элементов во входных массивов `distances` и `angles` отличаются, необходимо возбудить исключение `ShapeMismatchError`.

**Решение**:

In [23]:
class ShapeMismatchError(Exception):
    pass

In [ ]:
def convert_from_polar(
    distances: np.ndarray,
    angles: np.ndarray,
) -> tuple[np.ndarray, np.ndarray]:
    # код преподавателя

    return distances, angles


def convert_to_polar(
    abscissa: np.ndarray,
    ordinates: np.ndarray,
) -> tuple[np.ndarray, np.ndarray]:
    # код преподавателя

    return abscissa, ordinates

**Тестирование**:

In [29]:
abscissa = np.array([1, 0, -1, 0], dtype=np.float64)
ordinates = np.array([0, 1, 0, -1], dtype=abscissa.dtype)

distances, angles = convert_to_polar(abscissa, ordinates)
abscissa_conv, ordinates_conv = convert_from_polar(distances, angles)

assert np.allclose(abscissa, abscissa_conv)
assert np.allclose(ordinates, ordinates_conv)

## Задача 1. Долой Python! Да здравствует NumPy!

В этом задании вам будут предложены реализации некоторых функций на Python. Ваша задача - векторизовать код этих функций, используя NumPy.

### Часть 1. Сложение массивов

Векторизуйте код функции `sum_arrays_naive`.

*Входные данные*:
- `lhs` - одномерный массив чисел с плавающей точкой;
- `rhs` - одномерный массив чисел с плавающей точкой;

*Выходные данные*:
- Одномерный массив, элементы которого - суммы соответствующих элементов входных массивов.

*Сторонние эффекты*:
- Если входные массивы `rhs` и `lhs` имеют разное число элементов, необходимо возбудить исключение `ShapeMismatchError`.

**Python функция**:

In [30]:
def sum_arrays_naive(
    lhs: list[float],
    rhs: list[float],
) -> list[float]:
    if len(lhs) != len(rhs):
        raise ShapeMismatchError
    
    return [
        elem_lhs + elem_rhs for elem_lhs, elem_rhs in zip(lhs, rhs)
    ]

**Векторизованный код**:

In [31]:
def sum_arrays_vectorized(
    lhs: np.ndarray,
    rhs: np.ndarray,
) -> np.ndarray:
    # ваш код

    return np.array([])

### Часть 2. Полиномы

Векторизуйте код функции `compute_poly_naive`.

*Входные данные*:
- `abscissa` - одномерный массив чисел с плавающей точкой - область определения для вычисления полинома;

*Выходные данные*:
- Одномерный массив, элементы которого - значения полинома $y = 3 x^2 + 2x + 1$.

**Python функция**:

In [32]:
def compute_poly_naive(abscissa: list[float]) -> list[float]:
    return [3 * (x ** 2) + 2 * x + 1 for x in abscissa]

**Векторизованный код**:

In [33]:
def compute_poly_vectorized(abscissa: np.ndarray) -> np.ndarray:
    # ваш код

    return abscissa

### Часть 3. Далеко ли, близко ли?

Векторизуйте код функции `get_mutual_l2_distances_naive`.

*Входные данные*:
- `lhs` - двумерный массив чисел с плавающей точкой;
- `rhs` - двумерный массив чисел с плавающей точкой;

*Выходные данные*:
- Двумерный массив. Элемент `[i][j]` двумерного массива соответствует евклидову расстоянию между `i`-ым вектором из массива `lhs` и `j`-ым вектором из массива `rhs`. Под векторами подразумеваем строки входных двумерных массивов.

*Сторонние эффекты*:
- Если входные массивы `rhs` и `lhs` имеют разное число колонок, необходимо возбудить исключение `ShapeMismatchError`.

**Python функция**:

In [34]:
def get_mutual_l2_distances_naive(
    lhs: list[list[float]],
    rhs: list[list[float]],
) -> list[list[float]]:    
    if len(lhs[0]) != len(rhs[0]):
        raise ShapeMismatchError
    
    return [
        [
            sum(
                (lhs[i][k] - rhs[j][k]) ** 2 for k in range(len(lhs[0]))
            ) ** 0.5
            for j in range(len(rhs))
        ]
        for i in range(len(lhs))
    ]

**Векторизованный код**:

In [35]:
def get_mutual_l2_distances_vectorized(
    lhs: np.ndarray,
    rhs: np.ndarray,
) -> np.ndarray:
    # ваш код

    return np.zeros(shape=(lhs.shape[0], rhs.shape[0]))

## Задача 2. Сферические координаты: туда и обратно

Необходимо реализовать две функции: функцию перевода координат из трехмерной прямоугольной декартовой системы координат в сферические координаты и функцию перевода из сферических координат в трехмерную прямоугольную сферическую систему координат.

### Перевод из декартовых координат в сферические

*Входные данные*:
- `abscissa` - np.ndarray, абсциссы точек;
- `ordinates` - np.ndarray, ординаты точек;
- `applicates` - np.ndarray, аппликаты точек;

*Выходные данные*:
- Кортеж (`tuple`) из трех элементов, каждый элемент - np.ndarray. Первый элемент `tuple` - массив расстояний, второй элемент `tuple` - массив углов в диапазоне от $[-\pi, \pi]$ - углы азимута, третий элемент `tuple` - массив углов в диапазоне от $[-\pi, \pi]$ - углы места.

*Сторонние эффекты*:
- Если количество элементов во входных массивах `abscissa`, `ordinates` и `applicates` отличаются, необходимо возбудить исключение `ShapeMismatchError`.

*Замечания*:
- Гарантируется, что на вход подаются непустые одномерные массивы чисел с плавающей точкой.
- Предполагаем, что для перевода в сферические координаты используются следующие формулы:
    $$x = r*sin(\theta)*cos(\phi); y = r*sin(\theta)*sin(\phi);z = r*cos(\theta)$$

### Перевод из сферических координат в декартовы

*Входные данные*:
- `distances` - np.ndarray, массив расстояний;
- `azimuth` - np.ndarray, массив углов азимута в диапазоне $[-\pi, \pi]$;
- `inclination` - np.ndarray, массив углов места в диапазоне $[-\pi, \pi]$

*Выходные данные*:
- Кортеж (`tuple`) из трех элементов, каждый элемент - np.ndarray. Первый элемент `tuple` - массив абсцисс, второй элемент `tuple` - массив ординат, третий элемент `tuple` - массив аппликат.

*Сторонние эффекты*:
- Если количество элементов во входных массивов `distances`, `azimuth` и `inclination` отличаются, необходимо возбудить исключение `ShapeMismatchError`.

**Решение**:

In [36]:
def convert_from_sphere(
    distances: np.ndarray,
    azimuth: np.ndarray,
    inclination: np.ndarray,
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    # ваш код

    return distances, azimuth, inclination


def convert_to_sphere(
    abscissa: np.ndarray,
    ordinates: np.ndarray,
    applicates: np.ndarray,
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    # ваш код

    return abscissa, ordinates, applicates

## Задача 3. Низины и возвышенности

На вход подается одномерный массив чисел с плавающей точкой - значения некоторой функции на определенном отрезке. Ваша задача - вычислить индексы элементов, соответствующие точкам экстремума данной функции.

*Входные данные*:
- `ordinates` - np.ndarray числе с плавающей точкой, значения некоторой функции на определенном отрезке;

*Выходные данные*:
- Кортеж (`tuple`) из двух элементов. Элементы кортежа - np.ndarray. Первый элемент - индексы точек минимум, второй элемент - индексы точек максимума;

*Сторонние эффекты*:
- Если в массиве `ordinates` содержится менее трех элементов, необходимо возбудить `ValueError`.

*Замечение*:
- Краевые точки не принимают участия в вычислениях. Т.е. элементы с индексами 0 и -1 не могут быть точками экстремума.

**Решение**:

In [ ]:
def get_extremum_indices(
    ordinates: np.ndarray,
) -> tuple[np.ndarray, np.ndarray]:
    # ваш код

    return ordinates, ordinates

**Тестирование**:

In [39]:
ordinates = np.sin(2 * np.linspace(0, 4 * np.pi, 1000))
indices_min_expected = np.array([187, 437, 687, 937], dtype=np.int32)
indices_max_expected = np.array([ 62, 312, 562, 812], dtype=np.int32)

indices_min, indices_max = get_extremum_indices(ordinates)

assert np.allclose(indices_min, indices_min_expected)
assert np.allclose(indices_max, indices_max_expected)